<a href="https://colab.research.google.com/github/MatchLab-Imperial/keras_triplet_descriptor/blob/master/Baseline_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Necessary Modules

We now import the modules we will use in this baseline code. 

In [1]:
import sys
import json
import os
import glob
import time
import tensorflow as tf
import numpy as np
import cv2
import random

import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Reshape
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization 
from keras.layers import Input, UpSampling2D, concatenate  

from read_data import HPatches, DataGeneratorDesc, hpatches_sequence_folder, DenoiseHPatches, tps
from utils import generate_desc_csv, plot_denoise, plot_triplet

Using TensorFlow backend.


The `read_data` and `utils` imports are functions provided in the repository we just cloned. You can navigate through the *File tab* and check what those functions do for a better understanding.

![texto del enlace](https://i.ibb.co/HnfSvfT/filetab.png)





We also fix the seeds of the pseudo-random number generators to have reproducible results. The idea of fixing the seed is having the same results every time the algorithm is run if there are no changes in the code.

In [2]:
random.seed(1234)
np.random.seed(1234)
tf.set_random_seed(1234)

Now we load the data. The original HPatches dataset has several splits, which are used to separate the available sequences in train sequences and test sequences. For our experiments in N-HPatches we use the same splits as in HPatches. Specifically, we load (and report results) using the split `'a'`:


In [3]:
hpatches_dir = './hpatches'
splits_path = './splits.json'

splits_json = json.load(open(splits_path, 'rb'))
split = splits_json['a']

train_fnames = split['train']
test_fnames = split['test']

seqs = glob.glob(hpatches_dir+'/*')
seqs = [os.path.abspath(p) for p in seqs]   
seqs_train = list(filter(lambda x: x.split('\\')[-1] in train_fnames, seqs)) 
seqs_test = list(filter(lambda x: x.split('\\')[-1] in split['test'], seqs)) 
print(len(seqs_train))
print(len(seqs_test))

76
40


## Models and loss

We now define three functions that define the main modules of our baseline. 

*   **get_denoise_model(..)** returns the denoising model. The input for the function is the size of the patch, which will be *1x32x32*, and it outputs a keras denoising model. 
*   **get_descriptor_model(..)** builts the descriptor model. The input for the function is the size of the patch, which will be *1x32x32*, and it outputs a keras descriptor model. The model we use as baseline returns a descriptor of dimension *128x1*.
*   **triplet_loss(..)** defines the loss function which is used to train the descriptor model. 

You can modify the models in these functions and run the training code again. For example, the given denoising model is quite shallow, maybe using a deeper network can improve results. Or testing new initializations for the weights. Or maybe adding dropout. Or modifying the loss function somehow...

In [4]:
from keras.layers import  Input,Conv2D,BatchNormalization,Activation,Subtract
from keras.models import Model, load_model

def DnCNN(depth,filters=64,image_channels=1, use_bnorm=True):
  layer_count = 0
  inpt = Input(shape=(None,None,image_channels),name = 'input'+str(layer_count))
  # 1st layer, Conv+relu
  layer_count += 1
  x = Conv2D(filters=filters, kernel_size=(3,3), strides=(1,1),kernel_initializer='Orthogonal', padding='same',name = 'conv'+str(layer_count))(inpt)
  layer_count += 1
  x = Activation('relu',name = 'relu'+str(layer_count))(x)
  # depth-2 layers, Conv+BN+relu
  for i in range(depth-2):
    layer_count += 1
    x = Conv2D(filters=filters, kernel_size=(3,3), strides=(1,1),kernel_initializer='Orthogonal', padding='same',use_bias = False,name = 'conv'+str(layer_count))(x)
    if use_bnorm:
      layer_count += 1
      #x = BatchNormalization(axis=3, momentum=0.1,epsilon=0.0001, name = 'bn'+str(layer_count))(x)
    x = BatchNormalization(axis=3, momentum=0.0,epsilon=0.0001, name = 'bn'+str(layer_count))(x)
    layer_count += 1
    x = Activation('relu',name = 'relu'+str(layer_count))(x)
  # last layer, Conv
  layer_count += 1
  x = Conv2D(filters=image_channels, kernel_size=(3,3), strides=(1,1), kernel_initializer='Orthogonal',padding='same',use_bias = False,name = 'conv'+str(layer_count))(x)
  layer_count += 1
  x = Subtract(name = 'subtract' + str(layer_count))([inpt, x])   # input - noise
  model = Model(inputs=inpt, outputs=x)
  
  return model

def get_denoise_model(shape):
    
  inputs = Input(shape)
  
  ## Encoder starts
  conv1 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  
  ## Bottleneck
  conv2 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)

  ## Now the decoder starts
  up3 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv2))
  merge3 = concatenate([conv1,up3], axis = -1)
  conv3 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge3)
    
  conv4 = Conv2D(1, 3,  padding = 'same')(conv3)

  shallow_net = Model(inputs = inputs, outputs = conv4)
  
  #return shallow_net
  return DnCNN(depth=17,filters=64,image_channels=1,use_bnorm=True)



def get_descriptor_model(shape):
  
  '''Architecture copies HardNet architecture'''
  
  init_weights = keras.initializers.he_normal()
  
  descriptor_model = Sequential()
  descriptor_model.add(Conv2D(32, 3, padding='same', input_shape=shape, use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(32, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(64, 3, padding='same', strides=2, use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(64, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(128, 3, padding='same', strides=2,  use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(128, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))
  descriptor_model.add(Dropout(0.3))

  descriptor_model.add(Conv2D(128, 8, padding='valid', use_bias = True, kernel_initializer=init_weights))
  
  # Final descriptor reshape
  descriptor_model.add(Reshape((128,)))
  
  return descriptor_model
  
  
def triplet_loss(x):
  
  output_dim = 128
  a, p, n = x
  _alpha = 1.0
  positive_distance = K.mean(K.square(a - p), axis=-1)
  negative_distance = K.mean(K.square(a - n), axis=-1)
  
  return K.expand_dims(K.maximum(0.0, positive_distance - negative_distance + _alpha), axis = 1)

## Denoising Image Patches


We use the *DenoiseHPatches* class implemented in the read_data.py file, which takes as input the list of sequences to load and the size of batches. 

*DenoiseHPatches* outputs batches where the input data is the noisy image and the label is the clean image, so we can use a mean absolute error (MAE) metric as loss function. You can try to use different metrics here to see if that improves results. 

Afterward, we take a subset of training and validation sequences by using *random.sample* (3 sequences for training and 1 for validation data). The purpose of doing so is just to speed-up training when trying different setups, but you should use the whole dataset when training your final model. Remove the random.sample function to give the generator all the training data.

In addition, note that we are using the test set as validation. We will provide you with a new test set that will be used to evaluate your final model, and from which you will not have the clean images. 

**Updated**: Training should be quite faster now (1 epoch around 15 minutes).

In [5]:
#denoise_generator = DenoiseHPatches(random.sample(seqs_train, 3), batch_size=50)
#denoise_generator_val = DenoiseHPatches(random.sample(seqs_test, 1), batch_size=50)

# Uncomment following lines for using all the data to train the denoising model
denoise_generator = DenoiseHPatches(seqs_train, batch_size=50)
denoise_generator_val = DenoiseHPatches(seqs_test, batch_size=50)

100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:59<00:00,  1.31s/it]


In [6]:
shape = (32, 32, 1)
denoise_model = get_denoise_model(shape)

Instructions for updating:
Colocations handled automatically by placer.


We set number of epochs to 1, tweak it, along with other hyperparameters, to improve the performance of the model.

In [ ]:
sgd = keras.optimizers.SGD(lr=0.00001, momentum=0.9, nesterov=True)
adam = keras.optimizers.Adam(0.001)
denoise_model.compile(loss='mean_absolute_error', optimizer=adam, metrics=['mae'])
epochs = 1
### Use a loop to save for each epoch the weights in an external website in
### case colab stops. Every time you call fit/fit_generator the weigths are NOT
### reset, so e.g. calling 5 times fit(epochs=1) behave as fit(epochs=5)
for e in range(epochs):
  denoise_history = denoise_model.fit_generator(generator=denoise_generator, 
                                                epochs=1, 
                                                validation_data=denoise_generator_val)
  ### Saves optimizer and weights
  denoise_model.save('models/' + str(e) + 'epoch_denoise.h5') 
  ### Uploads files to external hosting
  #!curl -F "file=@denoise.h5" https://file.io


Instructions for updating:
Use tf.cast instead.
Epoch 1/1


  136/31179 [..............................] - ETA: 40:46:35 - loss: 16.5258 - mean_absolute_error: 16.525 - ETA: 20:50:38 - loss: 16.9574 - mean_absolute_error: 16.957 - ETA: 14:12:24 - loss: 16.6620 - mean_absolute_error: 16.662 - ETA: 10:52:08 - loss: 16.9243 - mean_absolute_error: 16.924 - ETA: 8:52:23 - loss: 16.6899 - mean_absolute_error: 16.689 - ETA: 7:32:28 - loss: 16.8235 - mean_absolute_error: 16.82 - ETA: 6:35:16 - loss: 16.6621 - mean_absolute_error: 16.66 - ETA: 5:52:16 - loss: 16.6095 - mean_absolute_error: 16.60 - ETA: 5:18:47 - loss: 16.5641 - mean_absolute_error: 16.56 - ETA: 4:52:05 - loss: 16.4185 - mean_absolute_error: 16.41 - ETA: 4:30:09 - loss: 16.3775 - mean_absolute_error: 16.37 - ETA: 4:11:44 - loss: 16.2562 - mean_absolute_error: 16.25 - ETA: 3:56:29 - loss: 16.1911 - mean_absolute_error: 16.19 - ETA: 3:43:20 - loss: 16.0561 - mean_absolute_error: 16.05 - ETA: 3:31:56 - loss: 16.0159 - mean_absolute_error: 16.01 - ETA: 3:21:50 - loss: 15.9753 - mean_absolute

  277/31179 [..............................] - ETA: 1:08:54 - loss: 10.1590 - mean_absolute_error: 10.15 - ETA: 1:08:46 - loss: 10.1340 - mean_absolute_error: 10.13 - ETA: 1:08:40 - loss: 10.1032 - mean_absolute_error: 10.10 - ETA: 1:08:32 - loss: 10.0759 - mean_absolute_error: 10.07 - ETA: 1:08:25 - loss: 10.0524 - mean_absolute_error: 10.05 - ETA: 1:08:18 - loss: 10.0308 - mean_absolute_error: 10.03 - ETA: 1:08:11 - loss: 10.0093 - mean_absolute_error: 10.00 - ETA: 1:08:04 - loss: 9.9872 - mean_absolute_error: 9.9872 - ETA: 1:07:57 - loss: 9.9598 - mean_absolute_error: 9.95 - ETA: 1:07:51 - loss: 9.9331 - mean_absolute_error: 9.93 - ETA: 1:07:44 - loss: 9.9069 - mean_absolute_error: 9.90 - ETA: 1:07:37 - loss: 9.8819 - mean_absolute_error: 9.88 - ETA: 1:07:31 - loss: 9.8564 - mean_absolute_error: 9.85 - ETA: 1:07:25 - loss: 9.8321 - mean_absolute_error: 9.83 - ETA: 1:07:18 - loss: 9.8099 - mean_absolute_error: 9.80 - ETA: 1:07:13 - loss: 9.7837 - mean_absolute_error: 9.78 - ETA: 1:07

  423/31179 [..............................] - ETA: 59:47 - loss: 8.0375 - mean_absolute_error: 8.03 - ETA: 59:45 - loss: 8.0286 - mean_absolute_error: 8.02 - ETA: 59:43 - loss: 8.0198 - mean_absolute_error: 8.01 - ETA: 59:41 - loss: 8.0081 - mean_absolute_error: 8.00 - ETA: 59:39 - loss: 7.9982 - mean_absolute_error: 7.99 - ETA: 59:37 - loss: 7.9922 - mean_absolute_error: 7.99 - ETA: 59:35 - loss: 7.9866 - mean_absolute_error: 7.98 - ETA: 59:33 - loss: 7.9766 - mean_absolute_error: 7.97 - ETA: 59:30 - loss: 7.9678 - mean_absolute_error: 7.96 - ETA: 59:29 - loss: 7.9591 - mean_absolute_error: 7.95 - ETA: 59:27 - loss: 7.9513 - mean_absolute_error: 7.95 - ETA: 59:25 - loss: 7.9429 - mean_absolute_error: 7.94 - ETA: 59:23 - loss: 7.9352 - mean_absolute_error: 7.93 - ETA: 59:21 - loss: 7.9276 - mean_absolute_error: 7.92 - ETA: 59:19 - loss: 7.9211 - mean_absolute_error: 7.92 - ETA: 59:17 - loss: 7.9127 - mean_absolute_error: 7.91 - ETA: 59:14 - loss: 7.9047 - mean_absolute_error: 7.90 - E

  569/31179 [..............................] - ETA: 56:27 - loss: 7.2173 - mean_absolute_error: 7.21 - ETA: 56:26 - loss: 7.2135 - mean_absolute_error: 7.21 - ETA: 56:26 - loss: 7.2089 - mean_absolute_error: 7.20 - ETA: 56:25 - loss: 7.2044 - mean_absolute_error: 7.20 - ETA: 56:24 - loss: 7.2009 - mean_absolute_error: 7.20 - ETA: 56:23 - loss: 7.1984 - mean_absolute_error: 7.19 - ETA: 56:22 - loss: 7.1940 - mean_absolute_error: 7.19 - ETA: 56:21 - loss: 7.1905 - mean_absolute_error: 7.19 - ETA: 56:20 - loss: 7.1859 - mean_absolute_error: 7.18 - ETA: 56:20 - loss: 7.1838 - mean_absolute_error: 7.18 - ETA: 56:19 - loss: 7.1808 - mean_absolute_error: 7.18 - ETA: 56:18 - loss: 7.1784 - mean_absolute_error: 7.17 - ETA: 56:17 - loss: 7.1739 - mean_absolute_error: 7.17 - ETA: 56:16 - loss: 7.1711 - mean_absolute_error: 7.17 - ETA: 56:16 - loss: 7.1676 - mean_absolute_error: 7.16 - ETA: 56:15 - loss: 7.1634 - mean_absolute_error: 7.16 - ETA: 56:14 - loss: 7.1609 - mean_absolute_error: 7.16 - E

...

 2905/31179 [=>............................] - ETA: 49:46 - loss: 5.5622 - mean_absolute_error: 5.56 - ETA: 49:46 - loss: 5.5622 - mean_absolute_error: 5.56 - ETA: 49:46 - loss: 5.5621 - mean_absolute_error: 5.56 - ETA: 49:46 - loss: 5.5618 - mean_absolute_error: 5.56 - ETA: 49:46 - loss: 5.5615 - mean_absolute_error: 5.56 - ETA: 49:46 - loss: 5.5613 - mean_absolute_error: 5.56 - ETA: 49:46 - loss: 5.5611 - mean_absolute_error: 5.56 - ETA: 49:46 - loss: 5.5609 - mean_absolute_error: 5.56 - ETA: 49:45 - loss: 5.5610 - mean_absolute_error: 5.56 - ETA: 49:45 - loss: 5.5608 - mean_absolute_error: 5.56 - ETA: 49:45 - loss: 5.5604 - mean_absolute_error: 5.56 - ETA: 49:45 - loss: 5.5604 - mean_absolute_error: 5.56 - ETA: 49:45 - loss: 5.5602 - mean_absolute_error: 5.56 - ETA: 49:45 - loss: 5.5599 - mean_absolute_error: 5.55 - ETA: 49:45 - loss: 5.5597 - mean_absolute_error: 5.55 - ETA: 49:45 - loss: 5.5595 - mean_absolute_error: 5.55 - ETA: 49:45 - loss: 5.5593 - mean_absolute_error: 5.55 - E

 3051/31179 [=>............................] - ETA: 49:33 - loss: 5.5402 - mean_absolute_error: 5.54 - ETA: 49:33 - loss: 5.5404 - mean_absolute_error: 5.54 - ETA: 49:32 - loss: 5.5403 - mean_absolute_error: 5.54 - ETA: 49:32 - loss: 5.5401 - mean_absolute_error: 5.54 - ETA: 49:32 - loss: 5.5399 - mean_absolute_error: 5.53 - ETA: 49:32 - loss: 5.5396 - mean_absolute_error: 5.53 - ETA: 49:32 - loss: 5.5394 - mean_absolute_error: 5.53 - ETA: 49:32 - loss: 5.5393 - mean_absolute_error: 5.53 - ETA: 49:32 - loss: 5.5393 - mean_absolute_error: 5.53 - ETA: 49:32 - loss: 5.5391 - mean_absolute_error: 5.53 - ETA: 49:32 - loss: 5.5388 - mean_absolute_error: 5.53 - ETA: 49:31 - loss: 5.5385 - mean_absolute_error: 5.53 - ETA: 49:31 - loss: 5.5384 - mean_absolute_error: 5.53 - ETA: 49:31 - loss: 5.5382 - mean_absolute_error: 5.53 - ETA: 49:31 - loss: 5.5380 - mean_absolute_error: 5.53 - ETA: 49:31 - loss: 5.5379 - mean_absolute_error: 5.53 - ETA: 49:31 - loss: 5.5376 - mean_absolute_error: 5.53 - E

 3197/31179 [==>...........................] - ETA: 49:18 - loss: 5.5201 - mean_absolute_error: 5.52 - ETA: 49:18 - loss: 5.5202 - mean_absolute_error: 5.52 - ETA: 49:18 - loss: 5.5200 - mean_absolute_error: 5.52 - ETA: 49:18 - loss: 5.5200 - mean_absolute_error: 5.52 - ETA: 49:18 - loss: 5.5199 - mean_absolute_error: 5.51 - ETA: 49:18 - loss: 5.5197 - mean_absolute_error: 5.51 - ETA: 49:17 - loss: 5.5196 - mean_absolute_error: 5.51 - ETA: 49:17 - loss: 5.5196 - mean_absolute_error: 5.51 - ETA: 49:17 - loss: 5.5193 - mean_absolute_error: 5.51 - ETA: 49:17 - loss: 5.5191 - mean_absolute_error: 5.51 - ETA: 49:17 - loss: 5.5188 - mean_absolute_error: 5.51 - ETA: 49:17 - loss: 5.5188 - mean_absolute_error: 5.51 - ETA: 49:17 - loss: 5.5186 - mean_absolute_error: 5.51 - ETA: 49:17 - loss: 5.5186 - mean_absolute_error: 5.51 - ETA: 49:17 - loss: 5.5184 - mean_absolute_error: 5.51 - ETA: 49:16 - loss: 5.5182 - mean_absolute_error: 5.51 - ETA: 49:16 - loss: 5.5180 - mean_absolute_error: 5.51 - E

 3343/31179 [==>...........................] - ETA: 49:02 - loss: 5.5018 - mean_absolute_error: 5.50 - ETA: 49:02 - loss: 5.5016 - mean_absolute_error: 5.50 - ETA: 49:02 - loss: 5.5012 - mean_absolute_error: 5.50 - ETA: 49:02 - loss: 5.5012 - mean_absolute_error: 5.50 - ETA: 49:02 - loss: 5.5010 - mean_absolute_error: 5.50 - ETA: 49:02 - loss: 5.5009 - mean_absolute_error: 5.50 - ETA: 49:02 - loss: 5.5008 - mean_absolute_error: 5.50 - ETA: 49:02 - loss: 5.5005 - mean_absolute_error: 5.50 - ETA: 49:01 - loss: 5.5004 - mean_absolute_error: 5.50 - ETA: 49:01 - loss: 5.5003 - mean_absolute_error: 5.50 - ETA: 49:01 - loss: 5.5003 - mean_absolute_error: 5.50 - ETA: 49:01 - loss: 5.5004 - mean_absolute_error: 5.50 - ETA: 49:01 - loss: 5.5002 - mean_absolute_error: 5.50 - ETA: 49:01 - loss: 5.5003 - mean_absolute_error: 5.50 - ETA: 49:01 - loss: 5.5001 - mean_absolute_error: 5.50 - ETA: 49:01 - loss: 5.5001 - mean_absolute_error: 5.50 - ETA: 49:01 - loss: 5.4998 - mean_absolute_error: 5.49 - E

 3476/31179 [==>...........................] - ETA: 48:47 - loss: 5.4818 - mean_absolute_error: 5.48 - ETA: 48:47 - loss: 5.4816 - mean_absolute_error: 5.48 - ETA: 48:46 - loss: 5.4814 - mean_absolute_error: 5.48 - ETA: 48:46 - loss: 5.4814 - mean_absolute_error: 5.48 - ETA: 48:46 - loss: 5.4814 - mean_absolute_error: 5.48 - ETA: 48:46 - loss: 5.4815 - mean_absolute_error: 5.48 - ETA: 48:46 - loss: 5.4813 - mean_absolute_error: 5.48 - ETA: 48:46 - loss: 5.4810 - mean_absolute_error: 5.48 - ETA: 48:46 - loss: 5.4810 - mean_absolute_error: 5.48 - ETA: 48:46 - loss: 5.4808 - mean_absolute_error: 5.48 - ETA: 48:46 - loss: 5.4806 - mean_absolute_error: 5.48 - ETA: 48:46 - loss: 5.4804 - mean_absolute_error: 5.48 - ETA: 48:46 - loss: 5.4802 - mean_absolute_error: 5.48 - ETA: 48:45 - loss: 5.4800 - mean_absolute_error: 5.48 - ETA: 48:45 - loss: 5.4799 - mean_absolute_error: 5.47 - ETA: 48:45 - loss: 5.4796 - mean_absolute_error: 5.47 - ETA: 48:45 - loss: 5.4794 - mean_absolute_error: 5.47 - E

After every epoch, the code will generate an external link, this link saves your weights in case of colab disconnecting during training. Example of an epoch:

**Epoch 1/1**
1797/1797 [==============================] - 48s 27ms/step - loss: 11.4135 - 
mean_absolute_error: 11.4135 - val_loss: 7.6013 - val_mean_absolute_error: 7.6013 
{"success":true,"key":"fv9vjj"

"link":"https://file.io/fv9vjj","expiry":"14 days"} **Epoch 1/1**

If colab did not disconnect, and you want to save the weights in your local disk, you also can use:


In [ ]:
#from google.colab import files
#files.download('denoise.h5')

Moreover, if you have a model saved from a previous training session, you can upload it to colab and initialize the model's weights with it. 

You either can use `!wget download_link` or upload the weights from your local disk by using the left panel ('Files' section) in colab.

Once the weights are uploaded, you can use

> ``denoise_model = keras.models.load_model('./denoise.h5')
``

to load the weights.

### Visualization of Denoising Results
To visualize how the denoised patches look, you can run the following function. It returns the noisy patch, the denoised patch in the middle, and the clean patch in the right side. 

In [ ]:
plot_denoise(denoise_model)

## Training a Descriptor Network
In the last section we trained a model that given a noisy patch, outputs a denoised version of it. We hoped that by doing so, we will improve the performance of the second part, which is training a network that outputs the descriptor. As we mentioned, a descriptor is a numerical vector that represents the small images we have. The dataset consists of a large number of small images, which are cropped patches from other larger images. Hence, they represent some local part of a scene. That is why there are no objects represented, only corners or textures. Each of these patches is related to a subset of other patches of the dataset by some kind of geometric transformation (e.g. rotation).  For a given patch, we want the network to output a vector that is close to the vectors of the patches that represent the same local part of a scene, while being far from patches do not represent that local part of a scene.

To do so, we will build a convolutional neural network that takes the input of $32\times32$ and outputs a descriptor of size $128$. For the loss, we use the triplet loss, which takes an anchor patch, a negative patch and a positive patch. The idea is to train the network so the descriptors from the anchor and positive patch have a low distance between them, and the negative and anchor patch have a large distance between them. 

In this cell we generate a triplet network, which is a network formed by three copies of the same network. That means that the descriptor model will compute the descriptor for the input `'a'` (anchor), the same descriptor model (with the same weights) will compute the descriptor for the input `'p'` (positive), and again the same model will compute the descriptor for the input `'n'` (negative). 

**Updated explanation**: Due to the way Keras handles the compile method, it needs a loss as an argument in that compile method. However, our loss is computed in the lambda layer, so we want to minimize the output of that layer. As we want to minimize the output of the Lambda function (in this case the triplet loss), we output as the label in the training_generator a vector of zeros and we compute the mean absolute error of the triplet loss and this vector of zeros. To give you an intuition, what we aim to minimize is
$$  |\text{triplet_loss} - 0| =  |\text{triplet_loss}| = \text{triplet_loss} $$



In [74]:
from keras.layers import Lambda
shape = (32, 32, 1)
xa = Input(shape=shape, name='a')
xp = Input(shape=shape, name='p')
xn = Input(shape=shape, name='n')
descriptor_model = get_descriptor_model(shape)
ea = descriptor_model(xa)
ep = descriptor_model(xp)
en = descriptor_model(xn)

loss = Lambda(triplet_loss)([ea, ep, en])

descriptor_model_trip = Model(inputs=[xa, xp, xn], outputs=loss)
sgd = keras.optimizers.SGD(lr=0.1)
descriptor_model_trip.compile(loss='mean_absolute_error', optimizer=sgd)

We now train the descriptor model and save the weights afterward.

In [46]:
epochs = 30
### As with the denoising model, we use a loop to save for each epoch 
## #the weights in an external website in case colab stops. 
### reset, so e.g. calling 5 times fit(epochs=1) behave as fit(epochs=5)

### If you have a model saved from a previous training session
### Load it in the next line
descriptor_model_trip.set_weights(keras.models.load_model('./descriptor.h5').get_weights())
descriptor_model_trip.optimizer = keras.models.load_model('./descriptor.h5').optimizer

for e in range(epochs):
  break
  
  #descriptor_history = descriptor_model_trip.fit_generator(generator=training_generator, epochs=1, verbose=1, validation_data=val_generator)
  
  ### Saves optimizer and weights
  #descriptor_model_trip.save('models/' + str(e) + 'epoch_descriptor.h5')
  ### Uploads files to external hosting
  #!curl -F "file=@descriptor.h5" https://file.io


Epoch 1/1


 313/2000 [===>..........................] - ETA: 2:23:38 - loss: 0.86 - ETA: 1:12:43 - loss: 0.62 - ETA: 49:07 - loss: 0.6143 - ETA: 37:18 - loss: 0.58 - ETA: 30:13 - loss: 0.54 - ETA: 25:28 - loss: 0.51 - ETA: 22:05 - loss: 0.50 - ETA: 19:32 - loss: 0.47 - ETA: 17:33 - loss: 0.47 - ETA: 15:58 - loss: 0.48 - ETA: 14:40 - loss: 0.48 - ETA: 13:35 - loss: 0.47 - ETA: 12:40 - loss: 0.47 - ETA: 11:53 - loss: 0.46 - ETA: 11:13 - loss: 0.47 - ETA: 10:37 - loss: 0.47 - ETA: 10:05 - loss: 0.48 - ETA: 9:37 - loss: 0.4765 - ETA: 9:12 - loss: 0.466 - ETA: 8:50 - loss: 0.467 - ETA: 8:29 - loss: 0.466 - ETA: 8:11 - loss: 0.460 - ETA: 7:54 - loss: 0.455 - ETA: 7:38 - loss: 0.460 - ETA: 7:24 - loss: 0.456 - ETA: 7:11 - loss: 0.465 - ETA: 6:59 - loss: 0.466 - ETA: 6:47 - loss: 0.459 - ETA: 6:37 - loss: 0.449 - ETA: 6:27 - loss: 0.441 - ETA: 6:18 - loss: 0.451 - ETA: 6:09 - loss: 0.450 - ETA: 6:01 - loss: 0.449 - ETA: 5:53 - loss: 0.448 - ETA: 5:46 - loss: 0.446 - ETA: 5:39 - loss: 0.441 - ETA: 5:32 - 

1999/2000 [============================>.] - ETA: 2s - loss: 0.219 - ETA: 2s - loss: 0.219 - ETA: 2s - loss: 0.219 - ETA: 2s - loss: 0.219 - ETA: 2s - loss: 0.219 - ETA: 2s - loss: 0.219 - ETA: 1s - loss: 0.218 - ETA: 1s - loss: 0.218 - ETA: 1s - loss: 0.218 - ETA: 1s - loss: 0.218 - ETA: 1s - loss: 0.218 - ETA: 1s - loss: 0.218 - ETA: 1s - loss: 0.218 - ETA: 1s - loss: 0.218 - ETA: 1s - loss: 0.218 - ETA: 1s - loss: 0.218 - ETA: 1s - loss: 0.218 - ETA: 1s - loss: 0.218 - ETA: 1s - loss: 0.218 - ETA: 1s - loss: 0.218 - ETA: 1s - loss: 0.218 - ETA: 1s - loss: 0.218 - ETA: 1s - loss: 0.218 - ETA: 1s - loss: 0.218 - ETA: 0s - loss: 0.218 - ETA: 0s - loss: 0.218 - ETA: 0s - loss: 0.218 - ETA: 0s - loss: 0.218 - ETA: 0s - loss: 0.218 - ETA: 0s - loss: 0.218 - ETA: 0s - loss: 0.218 - ETA: 0s - loss: 0.218 - ETA: 0s - loss: 0.218 - ETA: 0s - loss: 0.218 - ETA: 0s - loss: 0.217 - ETA: 0s - loss: 0.217 - ETA: 0s - loss: 0.217 - ETA: 0s - loss: 0.217 - ETA: 0s - loss: 0.217 - ETA: 0s - loss: 0.2

100%|███████████████████████████████████████████████████████████████████████| 100000/100000 [00:01<00:00, 59973.65it/s]


2000/2000 [==============================] - 118s 59ms/step - loss: 0.2178 - val_loss: 0.2473
Epoch 1/1


...

2000/2000 [==============================] - 113s 56ms/step - loss: 0.0790 - val_loss: 0.1143
Epoch 1/1


 314/2000 [===>..........................] - ETA: 1:53 - loss: 0.236 - ETA: 1:54 - loss: 0.120 - ETA: 1:51 - loss: 0.080 - ETA: 1:49 - loss: 0.060 - ETA: 1:49 - loss: 0.083 - ETA: 1:49 - loss: 0.071 - ETA: 1:48 - loss: 0.072 - ETA: 1:48 - loss: 0.063 - ETA: 1:47 - loss: 0.056 - ETA: 1:47 - loss: 0.062 - ETA: 1:48 - loss: 0.062 - ETA: 1:51 - loss: 0.061 - ETA: 1:52 - loss: 0.065 - ETA: 1:51 - loss: 0.067 - ETA: 1:51 - loss: 0.066 - ETA: 1:50 - loss: 0.069 - ETA: 1:50 - loss: 0.068 - ETA: 1:50 - loss: 0.066 - ETA: 1:50 - loss: 0.065 - ETA: 1:49 - loss: 0.063 - ETA: 1:49 - loss: 0.071 - ETA: 1:49 - loss: 0.068 - ETA: 1:49 - loss: 0.071 - ETA: 1:48 - loss: 0.076 - ETA: 1:48 - loss: 0.077 - ETA: 1:48 - loss: 0.075 - ETA: 1:48 - loss: 0.074 - ETA: 1:48 - loss: 0.075 - ETA: 1:48 - loss: 0.073 - ETA: 1:47 - loss: 0.071 - ETA: 1:47 - loss: 0.071 - ETA: 1:47 - loss: 0.072 - ETA: 1:47 - loss: 0.077 - ETA: 1:47 - loss: 0.075 - ETA: 1:47 - loss: 0.075 - ETA: 1:47 - loss: 0.073 - ETA: 1:47 - loss: 0

1999/2000 [============================>.] - ETA: 1s - loss: 0.075 - ETA: 1s - loss: 0.075 - ETA: 1s - loss: 0.075 - ETA: 1s - loss: 0.075 - ETA: 1s - loss: 0.075 - ETA: 1s - loss: 0.075 - ETA: 1s - loss: 0.075 - ETA: 1s - loss: 0.075 - ETA: 1s - loss: 0.075 - ETA: 1s - loss: 0.075 - ETA: 1s - loss: 0.075 - ETA: 1s - loss: 0.075 - ETA: 1s - loss: 0.075 - ETA: 1s - loss: 0.075 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.0755

100%|███████████████████████████████████████████████████████████████████████| 100000/100000 [00:01<00:00, 58432.29it/s]


2000/2000 [==============================] - 113s 56ms/step - loss: 0.0755 - val_loss: 0.1309


## Generating descriptors files for test data 

To evaluate the performance of out model we will use an existing evaluation code, which is called HPatches benchmark. HPatches benchmark takes as input the descriptors for the test data in a CSV form. So the whole pipeline is represented in the following image.

![](https://i.ibb.co/WcDDf3q/Screenshot-from-2019-02-15-11-17-24.png)

This function generates those files by passing it a descriptor model and a denoising model. It performs a first step of denoising the patches, and a second one of computing the descriptor of the denoised patch. If no denoising model is given (variable set to `None`), the descriptor is computed directly in the noisy patch.

Similarly to the loading data part, you have the denoise_model variable and `use_clean` variable. If `use_clean` is set to True, the CSV generated will be those of the clean patches, even if a denoising model is given. If set to False, then depends on the variable `denoise_model`. If there is no denoise model (`denoise_model=None`), then it will use the noisy patches. If you give a denoising model, then it will compute the CSV for the denoised patches. This can be useful to explore different scenarios (for example, the Upper Bound can be training the descriptor network with clean patches, and testing with clean patches), however you should always report the score when using noisy patches (depending on the approach you develop, you may want to denoise them or not). The official baseline uses the denoised patches. 

In [75]:
generate_desc_csv(descriptor_model, seqs_test, denoise_model=denoise_model, use_clean=False)




  0%|                                                                                           | 0/40 [00:00<?, ?it/s]


  2%|██                                                                                 | 1/40 [00:05<03:35,  5.53s/it]


  5%|████▏                                                                              | 2/40 [00:12<03:47,  5.98s/it]


  8%|██████▏                                                                            | 3/40 [00:14<02:59,  4.85s/it]


 10%|████████▎                                                                          | 4/40 [00:17<02:33,  4.27s/it]

KeyboardInterrupt: 

## Evaluating descriptors in HPatches Benchmark
We use HPatches benchmark code to compute the results for our model. 

**Updated**: The necessary code is included in the repository we cloned at the beginning of the code, so we do not need to download any extra data. Also, we simplified the results, so now they only return one value for each of the three tasks.

Now we will perform the evaluation of three different tasks (Verification, Matching and Evaluation) using the CSV files we generated as input and the `hpatches_eval.py` script. We also print the results using the `hpatches_results.py` script. The scripts will return a score for each of the tasks. The metric used is called mean Average Precision, which it uses the Precision of the model. The Precision is defined, for a given number of retrieved elements, as the ratio of correct retrieved elements / number of retrieved elements. [Link to Wikipedia with Precision explanation](https://en.wikipedia.org/wiki/Precision_and_recall). The definition of the three different tasks is taken from the [HPatches paper](https://arxiv.org/pdf/1704.05939.pdf).

In all of the tasks if you use the optional argument `--more_info` in `hpatches_results.py` you can see extra mAP information. However, the important score is the mAP score reported without this flag.

### Verification

Patch verification measures the ability of a descriptor to classify whether two patches are extracted from the same measurement. Now we compute the score of our architecture in this task.




In [65]:
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=verification --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=verification



>> Running HPatch evaluation for custom
>> Please wait, loading the descriptor files...


Traceback (most recent call last):
  File "./hpatches-benchmark/hpatches_eval.py", line 49, in <module>
    descr = load_descrs(path,dist=opts['--dist'],sep=opts['--delimiter'])
  File "C:\Users\koral\OneDrive - Imperial College London\University\Course\4th Year\Spring\EE3-25 Deep Learning\Coursework 2019\keras_triplet_descriptor\hpatches-benchmark\utils\hpatch.py", line 63, in load_descrs
    seqs['dim'] = seqs_l[0].dim
IndexError: list index out of range


Verification task results:


Traceback (most recent call last):
  File "./hpatches-benchmark/hpatches_results.py", line 35, in <module>
    results_methods[t](desc,splt,opts['--more_info'])
  File "C:\Users\koral\OneDrive - Imperial College London\University\Course\4th Year\Spring\EE3-25 Deep Learning\Coursework 2019\keras_triplet_descriptor\hpatches-benchmark\utils\results.py", line 12, in results_verification
    res = dill.load(open(os.path.join("results", desc+"_verification_"+splt['name']+".p"), "rb"))
FileNotFoundError: [Errno 2] No such file or directory: 'results\\custom_verification_a.p'


### Matching
Image matching, tests to what extent a descriptor can correctly identify correspondences in two images.

In [49]:
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=matching --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=matching


Traceback (most recent call last):
  File "./hpatches-benchmark/hpatches_eval.py", line 21, in <module>
    from utils.hpatch import *
  File "C:\Users\koral\OneDrive - Imperial College London\University\Course\4th Year\Spring\EE3-25 Deep Learning\Coursework 2019\keras_triplet_descriptor\hpatches-benchmark\utils\hpatch.py", line 4, in <module>
    from joblib import Parallel, delayed
ModuleNotFoundError: No module named 'joblib'
Traceback (most recent call last):
  File "./hpatches-benchmark/hpatches_results.py", line 19, in <module>
    from utils.tasks import tskdir
  File "C:\Users\koral\OneDrive - Imperial College London\University\Course\4th Year\Spring\EE3-25 Deep Learning\Coursework 2019\keras_triplet_descriptor\hpatches-benchmark\utils\tasks.py", line 4, in <module>
    from joblib import Parallel, delayed
ModuleNotFoundError: No module named 'joblib'


### Retrieval
Retrieval tests how well a descriptor can match a query patch to a pool of patches extracted from many images.

In [50]:
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=retrieval --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=retrieval

Traceback (most recent call last):
  File "./hpatches-benchmark/hpatches_eval.py", line 21, in <module>
    from utils.hpatch import *
  File "C:\Users\koral\OneDrive - Imperial College London\University\Course\4th Year\Spring\EE3-25 Deep Learning\Coursework 2019\keras_triplet_descriptor\hpatches-benchmark\utils\hpatch.py", line 4, in <module>
    from joblib import Parallel, delayed
ModuleNotFoundError: No module named 'joblib'
Traceback (most recent call last):
  File "./hpatches-benchmark/hpatches_results.py", line 19, in <module>
    from utils.tasks import tskdir
  File "C:\Users\koral\OneDrive - Imperial College London\University\Course\4th Year\Spring\EE3-25 Deep Learning\Coursework 2019\keras_triplet_descriptor\hpatches-benchmark\utils\tasks.py", line 4, in <module>
    from joblib import Parallel, delayed
ModuleNotFoundError: No module named 'joblib'


## Compressing and saving the CSV files 

This is not necessary for the analysis of the baseline code included in the report. However, we will be hosting a competition in an external website to see who can achieve the highest score. In that case, you will need to submit the CSV files, as the scoring script will be performed in an external server. With that aim, we include here a way to save the files either in your local disc or in your google drive account.

We first compress the directory with all the CSV by using the following command. Remove the `q` option if you want it to output the progress.

In [51]:
!zip -rq descriptors.zip ./out/custom


zip error: Nothing to do! (try: zip -rq descriptors.zip . -i ./out/custom)


The generated .zip is quite large, the method we used for the weights does not work. We have two other methods. First, in the file explorer in the left column we can right-click in the file and then click download. Then, we will see a circle next to the file showing the download progress.

The second way does not require for you to download the files, it save the zip file in your Google Drive account, and you can download it later to your machine if you want. To do so, follow this method (found [here](https://stackoverflow.com/questions/49428332/how-to-download-large-files-like-weights-of-a-model-from-colaboratory)). First run the next cell, and the output will be a link for authentication purposes, and just follow the instructions

In [0]:
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

auth.authenticate_user()
drive_service = build('drive', 'v3')

def save_file_to_drive(name, path):
  file_metadata = {
    'name': name,
    'mimeType': 'application/octet-stream'
  }

  media = MediaFileUpload(path, 
                          mimetype='application/octet-stream',
                          resumable=True)

  created = drive_service.files().create(body=file_metadata,
                                  media_body=media,
                                  fields='id').execute()

  print('File ID: {}'.format(created.get('id')))

  return created


Now we can use the following function to save the file to your drive account. The second argument is the name of the file we want to save, and the first argument the name that will have in your Drive.

In [4]:
shape = (32, 32, 1)
denoise_model = get_denoise_model(shape)
denoise_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input0 (InputLayer)             (None, None, None, 1 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 640         input0[0][0]                     
__________________________________________________________________________________________________
relu2 (Activation)              (None, None, None, 6 0           conv1[0][0]                      
__________________________________________________________________________________________________
conv3 (Conv2D)                  (None, None, None, 6 36864       relu2[0][0]                      
_____________________________________

In [5]:
from keras.utils.vis_utils import plot_model
plot_model(denoise_model, to_file='advanced_denoise_model_plot.png', show_shapes=True, show_layer_names=True)